In [1]:
import pandas as pd
import os

In [5]:
fpath = '/opt/dssg-hot/data/shirleydata/toll_prices_extra/xlsxfiles/'
files = os.listdir(fpath)
files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls_sorted = sorted(files_xls)

In [3]:
df = pd.DataFrame()

In [5]:
for f in files_xls_sorted:
    data = pd.read_excel(fpath+f, skiprows=4)
    df = df.append(data)
    print(f)

tmcRateFileDisplayReport_20180101.xlsx
tmcRateFileDisplayReport_20180102.xlsx
tmcRateFileDisplayReport_20180103.xlsx
tmcRateFileDisplayReport_20180104.xlsx
tmcRateFileDisplayReport_20180105.xlsx
tmcRateFileDisplayReport_20180106.xlsx
tmcRateFileDisplayReport_20180107.xlsx
tmcRateFileDisplayReport_20180108.xlsx
tmcRateFileDisplayReport_20180109.xlsx
tmcRateFileDisplayReport_20180110.xlsx
tmcRateFileDisplayReport_20180111.xlsx
tmcRateFileDisplayReport_20180112.xlsx
tmcRateFileDisplayReport_20180113.xlsx
tmcRateFileDisplayReport_20180114.xlsx
tmcRateFileDisplayReport_20180115.xlsx
tmcRateFileDisplayReport_20180116.xlsx
tmcRateFileDisplayReport_20180117.xlsx
tmcRateFileDisplayReport_20180118.xlsx
tmcRateFileDisplayReport_20180119.xlsx
tmcRateFileDisplayReport_20180120.xlsx
tmcRateFileDisplayReport_20180121.xlsx
tmcRateFileDisplayReport_20180122.xlsx
tmcRateFileDisplayReport_20180123.xlsx
tmcRateFileDisplayReport_20180124.xlsx
tmcRateFileDisplayReport_20180125.xlsx
tmcRateFileDisplayReport_

In [6]:
df.head()

,Fare File ID,Fare File Name,Trip Fare ID,Rate Sign Code,File Loaded Timestamp,Schedule Timestamp,Rate File End Date,Trip Fare effective Time,Trip Definition ID,Distance To Tollpoint,Feet Per Second,Sign Time,Effective Time Lag,Rate Increase,Trip Mode,Primary Rate
0,298286,20180101235848_TOLL.XML,11183953,405rs01375,2018-01-01 23:58:51,2018-01-01 23:59:08,2018-01-02 00:01:08,2018-01-01 23:59:41,2686,600.0,75,8.0,+000000000 00:00:33.904124000,NaN,I,NaN
1,298286,20180101235848_TOLL.XML,11183953,405rs01367,2018-01-01 23:58:51,2018-01-01 23:59:08,2018-01-02 00:01:08,2018-01-01 23:59:41,2686,600.0,75,8.0,+000000000 00:00:33.904124000,NaN,I,NaN
2,298286,20180101235848_TOLL.XML,11183953,405rs01364,2018-01-01 23:58:51,2018-01-01 23:59:08,2018-01-02 00:01:08,2018-01-01 23:59:41,2686,600.0,75,8.0,+000000000 00:00:33.904124000,NaN,I,NaN
3,298286,20180101235848_TOLL.XML,11183954,405rs01365,2018-01-01 23:58:51,2018-01-01 23:59:08,2018-01-02 00:01:08,2018-01-01 23:59:41,2687,600.0,75,8.0,+000000000 00:00:33.895928000,NaN,I,NaN
4,298286,20180101235848_TOLL.XML,11183954,405rs01368,2018-01-01 23:58:51,2018-01-01 23:59:08,2018-01-02 00:01:08,2018-01-01 23:59:41,2687,600.0,75,8.0,+000000000 00:00:33.895928000,NaN,I,NaN


In [10]:
# want only 3 cols
dfmod = df[['Trip Fare effective Time','Trip Definition ID','Primary Rate']]

In [11]:
# rename
dfmod = dfmod.rename(index=str, columns={'Trip Fare effective Time': 'Time', 'Primary Rate': 'Price'})

In [12]:
# want M-F
dfmod['weekday_num'] = dfmod['Time'].apply(lambda t: t.weekday())
dfmod = dfmod[dfmod['weekday_num'].between(0, 4, inclusive=True)]

In [13]:
# want 5am-8pm
dfmod['hour'] = dfmod['Time'].apply(lambda t: t.hour)
dfmod = dfmod[dfmod['hour'].between(5, 19, inclusive=True)]

In [14]:
dfmod1 = dfmod.sort_values(by='Time')

In [15]:
dfmod2 = dfmod1[['Time','Trip Definition ID','Price']]

In [16]:
dfmod3 = dfmod2.drop_duplicates(keep=False, inplace=False)

In [17]:
dfmod4 = dfmod3.pivot(index='Time', columns='Trip Definition ID', values='Price')

In [18]:
dfmod4.reset_index(level=0, inplace=True)

In [25]:
# The displayed toll xlsx files didn't have tolls for trip def IDs starting
# on direct access on ramps (NB2 = 4, NB6 = 8, SB7 = 19). But the tolls
# for these trips should be the same as if they entered at the toll gate
# just upstream of the ramp. The following is a key for associating the 
# direct access entry ramp trip def IDs to those of their upstream neighbors:
# 4-->3
# 2774 = 2736
# 2731 = 2735
# 2732 = 2734
# 8-->6
# 2733 = 2741
# 19-->17
# 2746 = 2748
dfmod4['2774'] = dfmod4['2736']
dfmod4['2731'] = dfmod4['2735']
dfmod4['2732'] = dfmod4['2734']
dfmod4['2733'] = dfmod4['2741']
dfmod4['2746'] = dfmod4['2748']

In [27]:
dfmod4.head()

,Time,2689,2690,2691,2692,2693,2694,2695,2696,2697,...,2769,2770,2771,2772,2773,2774,2731,2732,2733,2746
0,2018-01-01 05:01:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 05:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 05:02:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 05:02:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 05:04:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
dfmod4.to_csv(r'/opt/dssg-hot/data/toll_prices/concatenated_405_prices_2018_new.csv', index = None, header = True)